# Absolute Momentum
---
- Momentum: 물질의 운동량이나 가속도를 의미하는 용어, 주가가 방향성을 유지하려는 힘으로 통용
- 과거 시점 대비 현재 시점의 절대적 상승세를 평가한 전략
- 최근 N개월간 수익률이 양수(Positive)이면 매수하고 음수(Nagative)이면 공매도하는 전략 

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
yf.pdr_override() # that's all it takes
from pandas_datareader import data as pdr

In [37]:
read_df = pdr.get_data_yahoo('SPY')
read_df.reset_index(inplace=True)
read_df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.799770,1003200
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,25.983273,480500
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.038315,201300
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,26.313566,529400
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,26.423655,531500


In [38]:
price_df = read_df.loc[:,['Date','Adj Close']].copy()
price_df.head()

,Date,Adj Close
0,1993-01-29,25.799770
1,1993-02-01,25.983273
2,1993-02-02,26.038315
3,1993-02-03,26.313566
4,1993-02-04,26.423655


In [39]:
# Date 
# 월별 말 일자에 해당한는 종가 계산
price_df['Date'] = price_df['Date'].astype(str)
d_lam = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m')
price_df['STD_YM'] = price_df['Date'].map(d_lam)
price_df.head()

,Date,Adj Close,STD_YM
0,1993-01-29,25.799770,1993-01
1,1993-02-01,25.983273,1993-02
2,1993-02-02,26.038315,1993-02
3,1993-02-03,26.313566,1993-02
4,1993-02-04,26.423655,1993-02


In [40]:
month_list = price_df['STD_YM'].unique()
month_list

array(['1993-01', '1993-02', '1993-03', '1993-04', '1993-05', '1993-06',
       '1993-07', '1993-08', '1993-09', '1993-10', '1993-11', '1993-12',
       '1994-01', '1994-02', '1994-03', '1994-04', '1994-05', '1994-06',
       '1994-07', '1994-08', '1994-09', '1994-10', '1994-11', '1994-12',
       '1995-01', '1995-02', '1995-03', '1995-04', '1995-05', '1995-06',
       '1995-07', '1995-08', '1995-09', '1995-10', '1995-11', '1995-12',
       '1996-01', '1996-02', '1996-03', '1996-04', '1996-05', '1996-06',
       '1996-07', '1996-08', '1996-09', '1996-10', '1996-11', '1996-12',
       '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06',
       '1997-07', '1997-08', '1997-09', '1997-10', '1997-11', '1997-12',
       '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06',
       '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12',
       '1999-01', '1999-02', '1999-03', '1999-04', '1999-05', '1999-06',
       '1999-07', '1999-08', '1999-09', '1999-10', 

In [41]:
month_last_df = pd.DataFrame()
for m in month_list:
    # 기준년월에 맞는 인덱스의 가장 마지막 날짜 row릎 데이터 프레임에 추가
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1],:])

month_last_df.set_index(['Date'], inplace=True)
month_last_df.head()

,Adj Close,STD_YM
Date,,
1993-01-29,25.799770,1993-01
1993-02-26,26.075027,1993-02
1993-03-31,26.659071,1993-03
1993-04-30,25.976923,1993-04
1993-05-28,26.677525,1993-05


In [42]:
# 모멘텀 지수 계산을 위한 이전 시점의 데이터 가공
month_last_df['BF_1M_Adj Close'] = month_last_df.shift(1)['Adj Close']
month_last_df['BF_12M_Adj Close'] = month_last_df.shift(12)['Adj Close']
month_last_df.fillna(0, inplace=True)
month_last_df.head()

,Adj Close,STD_YM,BF_1M_Adj Close,BF_12M_Adj Close
Date,,,,
1993-01-29,25.799770,1993-01,0.000000,0.0
1993-02-26,26.075027,1993-02,25.799770,0.0
1993-03-31,26.659071,1993-03,26.075027,0.0
1993-04-30,25.976923,1993-04,26.659071,0.0
1993-05-28,26.677525,1993-05,25.976923,0.0


In [43]:
# 모멘텀 지수를 계산해서 거래가 생길 때 포지션을 기록할 DataFrame 생성
book = price_df.copy()
book.set_index(['Date'], inplace=True)
book['trade'] = ''
book.head()

,Adj Close,STD_YM,trade
Date,,,
1993-01-29,25.799770,1993-01,
1993-02-01,25.983273,1993-02,
1993-02-02,26.038315,1993-02,
1993-02-03,26.313566,1993-02,
1993-02-04,26.423655,1993-02,


- 월별 인덱스를 순회하면서 12개월 전 종가 대비 1개월 전 종가 수익률이 얼마인지 계산
- 계산된 수익률은 momentum_index 변수에 저장해 0 이상인지 확인
- 0 이상인 결우 모멘텀 현상이 나타난 것으로 판단해 매수 신호 발생(flag 처리)
- 월말 종가를 기준으로 매수/매도 신호를 계산하므로 최소 1개월 이상 해당 포지션 유지
- 포지션을 유지하는 기간은 모두 다르지만 보통 1개월을 유지함(revalance 주기)

In [44]:
# trading 부분
ticker = 'SPY'
for x in month_last_df.index:
    signal = ''
    # 절대 모멘텀을 계산
    momentum_index = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1
    # 절대 모멘텀 지표 True / False 판단
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False
    if flag:
        signal = 'buy ' + ticker # 절대 모멘텀 지표가 Positive이면 매수 후 보유 
    print('Date: ', x, 'Momentum Index: ', momentum_index, 'flag: ', flag, 'Signal: ', signal)
    book.loc[x:,'trade']  = signal

<ipython-input-44-2b8ee171b3c8>:6: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1
<ipython-input-44-2b8ee171b3c8>:6: RuntimeWarning: divide by zero encountered in double_scalars
  momentum_index = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1


Date:  1993-01-29 Momentum Index:  nan flag:  False Signal:  
Date:  1993-02-26 Momentum Index:  inf flag:  False Signal:  
Date:  1993-03-31 Momentum Index:  inf flag:  False Signal:  
Date:  1993-04-30 Momentum Index:  inf flag:  False Signal:  
Date:  1993-05-28 Momentum Index:  inf flag:  False Signal:  
Date:  1993-06-30 Momentum Index:  inf flag:  False Signal:  
Date:  1993-07-30 Momentum Index:  inf flag:  False Signal:  
Date:  1993-08-31 Momentum Index:  inf flag:  False Signal:  
Date:  1993-09-30 Momentum Index:  inf flag:  False Signal:  
Date:  1993-10-29 Momentum Index:  inf flag:  False Signal:  
Date:  1993-11-30 Momentum Index:  inf flag:  False Signal:  
Date:  1993-12-31 Momentum Index:  inf flag:  False Signal:  
Date:  1994-01-31 Momentum Index:  0.08709157055293648 flag:  True Signal:  buy SPY
Date:  1994-02-28 Momentum Index:  0.11312924517205403 flag:  True Signal:  buy SPY
Date:  1994-03-31 Momentum Index:  0.05699042951253386 flag:  True Signal:  buy SPY
Date

In [45]:
def returns(book, ticker):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == '': 
            buy = book.loc[i, 'Adj Close']
            print('진입일 : ', i, 'long 진입가격 : ', buy)
        
        # 보유중
        elif book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            current = book.loc[i, 'Adj Close']
            rtn = (current - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일: ', i, 'long 진입가격: ', buy, '  |  long 청산가격: ', sell, '  |  return: ', round(rtn, 4))
        
        # zero position
        if book.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
            current = 0.0
    
    acc_rtn = 1.0
    for i in book.index:
        # long 청산시
        if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            rtn = book.loc[i, 'return']
            acc_rtn = acc_rtn * rtn # 누적수익률 계산
            book.loc[i:, 'acc return'] = acc_rtn
    
    print('Accunulated return: ', round(acc_rtn, 4))
    return round(acc_rtn, 4)


In [46]:
returns(book, ticker)

진입일 :  1994-01-31 long 진입가격 :  29.02487564086914
청산일:  1994-12-30 long 진입가격:  29.02487564086914   |  long 청산가격:  28.15813446044922   |  return:  0.9701
진입일 :  1995-02-28 long 진입가격 :  30.292213439941406
청산일:  2000-12-29 long 진입가격:  30.292213439941406   |  long 청산가격:  88.8779296875   |  return:  2.934
진입일 :  2001-02-28 long 진입가격 :  83.97460174560547
청산일:  2001-03-30 long 진입가격:  83.97460174560547   |  long 청산가격:  79.2689208984375   |  return:  0.944
진입일 :  2003-07-31 long 진입가격 :  69.80838775634766
청산일:  2008-02-29 long 진입가격:  69.80838775634766   |  long 청산가격:  101.98289489746094   |  return:  1.4609
진입일 :  2009-10-30 long 진입가격 :  82.24986267089844
청산일:  2011-10-31 long 진입가격:  82.24986267089844   |  long 청산가격:  103.66520690917969   |  return:  1.2604
진입일 :  2011-11-30 long 진입가격 :  103.24393463134766
청산일:  2012-01-31 long 진입가격:  103.24393463134766   |  long 청산가격:  109.16056060791016   |  return:  1.0573
진입일 :  2012-02-29 long 진입가격 :  113.89874267578125
청산일:  2015-10-30 long 진입가격:  113.89874

11.7466

In [47]:
book.loc['1994-11-31':,]

,Adj Close,STD_YM,trade,return,acc return
Date,,,,,
1994-12-01,27.676758,1994-12,buy SPY,0.953553,NaN
1994-12-02,27.935396,1994-12,buy SPY,0.962464,NaN
1994-12-05,27.964149,1994-12,buy SPY,0.963455,NaN
1994-12-06,27.983311,1994-12,buy SPY,0.964115,NaN
1994-12-07,27.782125,1994-12,buy SPY,0.957183,NaN
...,...,...,...,...,...
2021-08-13,445.920013,2021-08,buy SPY,1.493669,11.746618
2021-08-16,446.970001,2021-08,buy SPY,1.497186,11.746618
2021-08-17,444.040009,2021-08,buy SPY,1.487371,11.746618


## Summary
---

In [58]:
read_df = pdr.get_data_yahoo('SPY')
read_df.reset_index(inplace=True)

price_df = read_df.loc[:,['Date','Adj Close']].copy()

# Date 
price_df['Date'] = price_df['Date'].astype(str)
d_lam = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m')
price_df['STD_YM'] = price_df['Date'].map(d_lam)

month_list = price_df['STD_YM'].unique()

month_last_df = pd.DataFrame()
for m in month_list:
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1],:])
    
month_last_df.set_index(['Date'],inplace=True)
month_last_df['BF_1M_Adj Close'] = month_last_df.shift(1)['Adj Close']
month_last_df['BF_12M_Adj Close'] = month_last_df.shift(12)['Adj Close']
month_last_df.fillna(0, inplace=True)

book = price_df.copy()
book.set_index(['Date'], inplace=True)
book['trade'] = ''

#trading 부분.
ticker = 'SPY'
for x in month_last_df.index:
    signal = ''
    momentum_index = month_last_df.loc[x,'BF_1M_Adj Close'] / month_last_df.loc[x,'BF_12M_Adj Close'] -1
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf))\
    else False \
    and True
    if flag :
        signal = 'buy ' + ticker
    print('날짜 : ',x,' 모멘텀 인덱스 : ',momentum_index, 'flag : ',flag ,'signal : ',signal)
    book.loc[x:,'trade'] = signal
    
returns(book,ticker)

[*********************100%***********************]  1 of 1 completed


<ipython-input-58-668dc6b9d00e>:30: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = month_last_df.loc[x,'BF_1M_Adj Close'] / month_last_df.loc[x,'BF_12M_Adj Close'] -1
<ipython-input-58-668dc6b9d00e>:30: RuntimeWarning: divide by zero encountered in double_scalars
  momentum_index = month_last_df.loc[x,'BF_1M_Adj Close'] / month_last_df.loc[x,'BF_12M_Adj Close'] -1


날짜 :  1993-01-29  모멘텀 인덱스 :  nan flag :  False signal :  
날짜 :  1993-02-26  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-03-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-04-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-05-28  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-06-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-07-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-08-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-09-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-10-29  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-11-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-12-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1994-01-31  모멘텀 인덱스 :  0.08709157055293648 flag :  True signal :  buy SPY
날짜 :  1994-02-28  모멘텀 인덱스 :  0.11312924517205403 flag :  True signal :  buy SPY
날짜 :  1994-03-31  모멘텀 인덱스 :  0.05699042951253386 flag :  True signal :  buy SPY
날짜 :  1994-04-29  모멘텀 인덱스 :  0.03929434719868863 flag :  True si

11.7466